In [111]:
import pandas as pd #importing essential library 
import numpy as np 

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from sklearn.metrics import mean_absolute_error


In [2]:
#renamed the files to make it more accessible and intuitive.
#reading csv files into pd dataframes for manipulation and data wrangling
cardholder_acc = pd.read_csv('../csvs_datathon/accy_dim.csv')
statement_data = pd.read_csv('../csvs_datathon/statement_fact.csv')
transaction_data = pd.read_csv('../csvs_datathon/transaction_fact.csv')
wrld_transaction_data = pd.read_csv('../csvs_datathon/wrld_stor_tran_fact.csv')
customer_id = pd.read_csv('../csvs_datathon/syf_id.csv')
acc_lvl_features = pd.read_csv('../csvs_datathon/rams_batch_cur.csv')
fraud_claim_case = pd.read_csv('../csvs_datathon/fraud_claim_case.csv')
fraud_claim_tran = pd.read_csv('../csvs_datathon/fraud_claim_tran.csv')

In [28]:
transaction_data_sales = transaction_data[(transaction_data['transaction_type'] != 'PAYMENT')]
wrld_transaction_data_sales = wrld_transaction_data[(wrld_transaction_data['transaction_type'] != "PAYMENT")] 

transaction_data_sales['transaction_date'] = pd.to_datetime(transaction_data_sales['transaction_date'])
transaction_data_salesf = transaction_data_sales[(transaction_data_sales['transaction_date'] >= "2024-08-01") & (transaction_data_sales['transaction_date'] <= "2025-03-31")] 

wrld_transaction_data_sales['transaction_date'] = pd.to_datetime(wrld_transaction_data_sales['transaction_date'])
wrld_transaction_data_salesf = wrld_transaction_data_sales[(wrld_transaction_data_sales['transaction_date'] >= "2024-08-01") & (wrld_transaction_data_sales['transaction_date'] <= "2025-03-31")]


total_sales_customer1 = transaction_data_sales.groupby('current_account_nbr')['transaction_amt'].sum().reset_index()
total_sales_customer1.rename(columns = {'transaction_amt' : 'total spent'})

total_sales_customer2 = wrld_transaction_data_sales.groupby('current_account_nbr')['transaction_amt'].sum().reset_index() 
total_sales_customer2.rename(columns = {'transaction_amt' : 'total spent'})

f_sales1 = transaction_data_salesf.groupby('current_account_nbr')['transaction_amt'].sum().reset_index()
f_sales2 = wrld_transaction_data_salesf.groupby('current_account_nbr')['transaction_amt'].sum().reset_index()

all_transactions = pd.DataFrame(columns = ['current account nbr', 'total spent'])
acc_nbrs = total_sales_customer1['current_account_nbr'].to_list() + total_sales_customer2['current_account_nbr'].to_list()
acc_total_spent = total_sales_customer1['transaction_amt'].to_list() + total_sales_customer2['transaction_amt'].to_list()


all_transactions['current_account_nbr'] = acc_nbrs
all_transactions['total spent'] = acc_total_spent

all_transactions['total spent per month'] = all_transactions['total spent'] / 12
all_transactions['total spent per quarter'] = all_transactions['total spent'] / 3
all_transactions

f_sales1.rename(columns = {'transaction_amt': 'total spent last 8 months'}, inplace = True)
f_sales2.rename(columns = {'transaction_amt': 'total spent last 8 months'}, inplace = True)
all_transactions = pd.merge(all_transactions, f_sales1, on = 'current_account_nbr', how = 'left')
all_transactions = pd.merge(all_transactions, f_sales2, on = 'current_account_nbr', how = 'left', suffixes = ('', '_wrld'))
all_transactions['total spent last 8 months'] = all_transactions[['total spent last 8 months', 'total spent last 8 months_wrld']].sum(axis=1)
all_transactions.drop(columns=['total spent last 8 months_wrld'], inplace = True)
all_transactions['total spent last 8 months'].fillna(0, inplace = True)

acc_lvl_features = acc_lvl_features.rename(columns = {'cu_account_nbr' : 'current_account_nbr'})
avg_util_3_months = acc_lvl_features.groupby('current_account_nbr')['ca_avg_utilz_lst_3_mnths'].mean().reset_index()
all_transactions = pd.merge(all_transactions, avg_util_3_months, on = 'current_account_nbr', how = 'left')
all_transactions['ca_avg_utilz_lst_3_mnths'].fillna(0, inplace = True)

acc_lvl_features['difference_behav_score'] = acc_lvl_features['rb_new_bhv_scr'] - acc_lvl_features['cu_bhv_scr']
diff_behav_scr = acc_lvl_features.groupby('current_account_nbr')['difference_behav_score'].mean().reset_index()
all_transactions = pd.merge(all_transactions, diff_behav_scr, on = 'current_account_nbr', how = 'left')
all_transactions['difference_behav_score'].fillna(0, inplace = True)
all_transactions = all_transactions.rename(columns = {'total spent' : 'total spent year'})

C:\Users\lyons\AppData\Local\Temp\ipykernel_18832\2553155581.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_data_sales['transaction_date'] = pd.to_datetime(transaction_data_sales['transaction_date'])
C:\Users\lyons\AppData\Local\Temp\ipykernel_18832\2553155581.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrld_transaction_data_sales['transaction_date'] = pd.to_datetime(wrld_transaction_data_sales['transaction_date'])


In [29]:
all_transactions

,current account nbr,total spent year,current_account_nbr,total spent per month,total spent per quarter,total spent last 8 months,ca_avg_utilz_lst_3_mnths,difference_behav_score
0,NaN,25.90,00gwujJkd597VxLd,2.158333,8.633333,25.90,0.000000,0.000000
1,NaN,4930.46,00iP5U82D8XwVQ9G,410.871667,1643.486667,3938.03,14.800000,0.000000
2,NaN,929.82,00oyr3QppAzjLws4,77.485000,309.940000,394.26,89.666667,2.166667
3,NaN,846.42,02WRVV8yusgcZWLW,70.535000,282.140000,10659.53,22.500000,-13.000000
4,NaN,2648.78,033o9yHYen3xoz6k,220.731667,882.926667,18048.16,14.333333,-2.000000
...,...,...,...,...,...,...,...,...
18721,NaN,6212.50,zyZhjzJwhpOgSvmc,517.708333,2070.833333,5234.37,38.333333,0.833333
18722,NaN,0.00,zz3nbtZXS41NZk0h,0.000000,0.000000,240.92,0.000000,122.166667
18723,NaN,2878.28,zzBy2qNM78aRV580,239.856667,959.426667,-0.24,3.000000,2.000000
18724,NaN,20413.63,zzEuUBBmvGiVnabb,1701.135833,6804.543333,16728.04,12.000000,-1.166667


In [73]:
#https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor
x = all_transactions[['total spent year', 'total spent per month', 'total spent last 8 months', 'ca_avg_utilz_lst_3_mnths', 'difference_behav_score']]
y = all_transactions[['total spent per quarter']]

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.20, random_state = 50)
xgboost_model = XGBRegressor(n_estimators = 200, learning_rate = 0.1)
xgboost_model.fit(X_train, Y_train)
predictions = xgboost_model.predict(X_test)

In [113]:
r_squared = r2_score(Y_test, predictions)
r_squared

0.9734937402902551

In [114]:
rmse = np.sqrt(mean_squared_error(Y_test, predictions))
rmse

683.8087572002423

In [103]:
#https://medium.com/@rithpansanga/optimizing-xgboost-a-guide-to-hyperparameter-tuning-77b6e48e289d
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}
random_search = RandomizedSearchCV(xgboost_model, param_distributions = param_dist, n_iter = 30, cv = 5, scoring = 'accuracy')
random_search.fit(X_train, Y_train) 

C:\Users\lyons\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lyons\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lyons\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000271CD278590>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000271CD2909D0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000271CD21D310>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000271CE193610>},
                   scoring='accuracy')

In [110]:
random_search.best_params_

{'learning_rate': 0.08866319931044156,
 'max_depth': 9,
 'n_estimators': 50,
 'subsample': 0.8036166601891328}

In [105]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(x, y, test_size = 0.20, random_state = 50)
xgboost_model = XGBRegressor(n_estimators = 50, learning_rate = 0.08866319931044156, max_depth = 9, subsample = 0.8036166601891328)
xgboost_model.fit(X_train2, Y_train2)
predictions2 = xgboost_model.predict(X_test2)

In [106]:
r_squared2 = r2_score(Y_test2, predictions2)
r_squared2

0.9778489542966003

In [107]:
rmse2 = np.sqrt(mean_squared_error(Y_test2, predictions2))
rmse2

625.1115890274757

In [108]:
range_y2 = Y_test2.max() - Y_test2.min()
n_rmse2 = rmse2 / range_y2
n_rmse2

total spent per quarter    0.009293
dtype: float64

In [112]:
mae = mean_absolute_error(Y_test2, predictions2)
mae

72.74572397546892